In [118]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import sqlite3 as sql
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler


database = "../KAGGLE/KAGGLE_PRUBAS_DATA/DATA/diamonds_train.db"
connection = sql.connect(database)

query = '''SELECT tr.index_id, c.clarity, cl.color, ct.cut, dm.depth, dm.'table', dm.x, dm.y, dm.z, tr.price, ci.city, tr.carat
FROM diamonds_properties pr
   INNER JOIN diamonds_clarity c ON pr.clarity_id = c.clarity_id
   INNER JOIN diamonds_color cl on pr.color_id = cl.color_id
   INNER JOIN diamonds_cut ct on pr.cut_id = ct.cut_id
   INNER JOIN diamonds_dimensions dm on pr.index_id = dm.index_id
   INNER JOIN diamonds_transactional tr on pr.index_id = tr.index_id
   INNER JOIN diamonds_city ci on tr.city_id = ci.city_id'''

diamonds = pd.read_sql_query(query,connection)





In [119]:
diamonds.head()

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02


In [120]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.05)
    q3 = df_in[col_name].quantile(0.95)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [121]:
diamonds =remove_outlier(diamonds,"depth")
diamonds =remove_outlier(diamonds,"table")
diamonds =remove_outlier(diamonds,"x")
diamonds=remove_outlier(diamonds,"y")
diamonds =remove_outlier(diamonds,"z")
diamonds=remove_outlier(diamonds,"carat")

In [122]:
diamonds.describe()

,depth,table,x,y,z,price,carat
count,40426.000000,40426.000000,40426.000000,40426.000000,40426.000000,40426.000000,40426.000000
mean,61.750918,57.443311,5.728657,5.730911,3.536448,3926.733439,0.797180
std,1.408720,2.215827,1.123584,1.115002,0.696046,3990.022788,0.474116
min,52.700000,44.000000,0.000000,0.000000,0.000000,326.000000,0.200000
25%,61.000000,56.000000,4.710000,4.720000,2.910000,945.000000,0.400000
50%,61.800000,57.000000,5.690000,5.710000,3.520000,2396.000000,0.700000
75%,62.500000,59.000000,6.540000,6.540000,4.030000,5330.750000,1.040000
max,70.500000,70.000000,9.660000,9.630000,6.270000,18823.000000,3.510000


In [123]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['J', 'I', 'H', 'G', 'F', 'E', 'D']])
encoder.fit(diamonds[["color"]])
diamonds["color-encoded"] = encoder.transform(diamonds[["color"]])
diamonds.drop("color", axis = 1 , inplace = True)

In [124]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1','IF']])
encoder.fit(diamonds[["clarity"]])
diamonds["clarity-encoded"] = encoder.transform(diamonds[["clarity"]])
diamonds.drop("clarity", axis = 1 , inplace = True)

In [125]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']])
encoder.fit(diamonds[["cut"]])
diamonds["cut-encoded"] = encoder.transform(diamonds[["cut"]])
diamonds.drop("cut", axis = 1 , inplace = True)

In [126]:
features = ['depth',
 'table',
 'x',
 'y',
 'z',
 'carat',
 "color-encoded",
 "clarity-encoded",
 "cut-encoded"
]

In [127]:
diamonds = diamonds.drop(columns = ["index_id"])

In [128]:
scaler = RobustScaler()
X = scaler.fit_transform(diamonds[features])
y = diamonds["price"]

In [129]:
 X_train, X_validacion, y_train, y_validacion = train_test_split(X, y, test_size=0.2, random_state=42)

In [130]:
print(f"X_train: {X_train.shape}, X_test: {X_validacion.shape}, y_train: {y_train.shape}")

X_train: (32340, 9), X_test: (8086, 9), y_train: (32340,)


In [131]:
#MODEL

#regressor = LinearRegression()
regressor = RandomForestRegressor(random_state= 42)
hyperparameters = regressor.get_params()
regressor.fit(X_train, y_train)
y_pred_train = regressor.predict(X_validacion)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
print('Ground truth target:', y_validacion, '\n')
print('Predicted target:', y_pred_train, '\n')

Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

Ground truth target: 15580     645
20731     720
24953     680
19363     650
27016     571
         ... 
21012     596
23693    1709
17946    8316
14722    9667
9428     3634
Name: price, Length: 8086, dtype: int64 

Predicted target: [ 641.08  698.48  664.79 ... 8533.62 8779.98 4202.89] 



In [132]:
RMSE_ERROR_sklearn = mean_squared_error(y_validacion,y_pred_train, squared = False)
print("TENEMOS UN ERROR DEL " + str(RMSE_ERROR_sklearn) + "%")

TENEMOS UN ERROR DEL 547.7230090128849%


APLICAMOS LO MISMO CON EL TEST

In [133]:
diamonds_test = pd.read_csv("../KAGGLE/KAGGLE_PRUBAS_DATA/DATA/diamonds_test.csv")
diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


In [134]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['J', 'I', 'H', 'G', 'F', 'E', 'D']])
encoder.fit(diamonds_test[["color"]])
diamonds_test["color-encoded"] = encoder.transform(diamonds_test[["color"]])
diamonds_test.drop("color", axis = 1 , inplace = True)

In [135]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1','IF']])
encoder.fit(diamonds_test[["clarity"]])
diamonds_test["clarity-encoded"] = encoder.transform(diamonds_test[["clarity"]])
diamonds_test.drop("clarity", axis = 1 , inplace = True)

In [136]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']])
encoder.fit(diamonds_test[["cut"]])
diamonds_test["cut-encoded"] = encoder.transform(diamonds_test[["cut"]])
diamonds_test.drop("cut", axis = 1 , inplace = True)

In [137]:
scaler = RobustScaler()
diamonds_test2 = scaler.fit_transform(diamonds_test[features])

In [138]:
y_pred = regressor.predict(diamonds_test2).clip(0,30000)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')

Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 



In [139]:
submission = pd.DataFrame({'id': diamonds_test["id"] , 'price': y_pred})
submission.shape

(13485, 2)

In [140]:
submission.to_csv("../KAGGLE/KAGGLE_PRUBAS_DATA/DATA/submission_final3.csv", index=False)